In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
from tqdm import tqdm

import itertools
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [9]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')

# 파생변수 생성 및 인코딩

In [10]:
## 나이 양수로 변환
train['birth_plus'] = train.DAYS_BIRTH*(-1)/365
test['birth_plus'] = test.DAYS_BIRTH*(-1)/365

train['gender']=LabelEncoder().fit_transform(train.gender)
train['car']=LabelEncoder().fit_transform(train.car)
train['reality']=LabelEncoder().fit_transform(train.reality)
train['income_type']=LabelEncoder().fit_transform(train.income_type)
train['edu_type']=LabelEncoder().fit_transform(train.edu_type)
train['family_type']=LabelEncoder().fit_transform(train.family_type)
train['house_type']=LabelEncoder().fit_transform(train.house_type)
train.fillna('NAN',inplace=True)
train['occyp_type']=LabelEncoder().fit_transform(train.occyp_type)
train['income_weight']=train.income_total**2
train['income_age']=train.income_total*train.DAYS_BIRTH
train['income_emp']=train.income_total*train.DAYS_EMPLOYED

test['gender']=LabelEncoder().fit_transform(test.gender)
test['car']=LabelEncoder().fit_transform(test.car)
test['reality']=LabelEncoder().fit_transform(test.reality)
test['income_type']=LabelEncoder().fit_transform(test.income_type)
test['edu_type']=LabelEncoder().fit_transform(test.edu_type)
test['family_type']=LabelEncoder().fit_transform(test.family_type)
test['house_type']=LabelEncoder().fit_transform(test.house_type)
test.fillna('NAN',inplace=True)
test['occyp_type']=LabelEncoder().fit_transform(test.occyp_type)
test['income_weight']=test.income_total**2
test['income_age']=test.income_total*test.DAYS_BIRTH
test['income_emp']=test.income_total*test.DAYS_EMPLOYED

train.income_total=train.income_total.astype(int)
train.family_size=train.family_size.astype(int)
train.begin_month=train.begin_month.astype(int)
train.birth_plus=train.birth_plus.astype(int)

test.income_total=test.income_total.astype(int)
test.family_size=test.family_size.astype(int)
test.begin_month=test.begin_month.astype(int)
test.birth_plus=test.birth_plus.astype(int)

train.credit=train.credit.astype('category')
train.drop(['index','family_size','work_phone'],axis=1,inplace=True)
test.drop(['index','family_size','work_phone'],axis=1,inplace=True)
train.DAYS_BIRTH=train.DAYS_BIRTH**6
test.DAYS_BIRTH=test.DAYS_BIRTH**6
train.income_type=1/(train.income_type)
test.income_type=1/(test.income_type)
train.income_weight=1/(train.income_weight)**3
test.income_weight=1/(test.income_weight)**3
train['DAYS_BIRTH_weight']=1/(train.DAYS_BIRTH)**2
test['DAYS_BIRTH_weight']=1/(test.DAYS_BIRTH)**2

In [21]:
## predict_rf
ftr = train.drop('credit', axis=1).values
target = train['credit'].values
tst_ar = test.values
n_class = 3
n_fold = 17

In [29]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=40)

print(ftr.shape)

xgb_p_val = np.zeros((ftr.shape[0], n_class))
xgb_p_tst = np.zeros((tst_ar.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(ftr, target), 1):
    print(f'training model for CV #{i}')
    xgb_clf = xgb.XGBClassifier(max_depth=10,
                                colsample_bytree=0.5,
                                colsample_bylevel=0.8,
                                colsample_bynode=0.5,
                                min_child_weight=0.5,
                                gamma=0,
                                max_delta_step=0,
                                subsample=0.9,
                                reg_alpha=0.09,
                                reg_lambda=0.6,
                                n_estimators=50,
                               random_state = 2021)

    xgb_clf.fit(ftr[i_trn], target[i_trn],
            eval_set=[(ftr[i_val], target[i_val])])
    
    xgb_p_val[i_val, :] = xgb_clf.predict_proba(ftr[i_val])
    xgb_p_tst += xgb_clf.predict_proba(tst_ar) / n_fold


print(f'{log_loss(target, xgb_p_val)}')
print(f'{confusion_matrix(target, np.argmax(xgb_p_val, axis=1))}%')

(26457, 21)
training model for CV #1
[16:22:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:0.98267
[1]	validation_0-mlogloss:0.91076
[2]	validation_0-mlogloss:0.86451
[3]	validation_0-mlogloss:0.83703
[4]	validation_0-mlogloss:0.81137
[5]	validation_0-mlogloss:0.79857
[6]	validation_0-mlogloss:0.78182
[7]	validation_0-mlogloss:0.77165
[8]	validation_0-mlogloss:0.76357
[9]	validation_0-mlogloss:0.75511
[10]	validation_0-mlogloss:0.75055
[11]	validation_0-mlogloss:0.74369
[12]	validation_0-mlogloss:0.74120
[13]	validation_0-mlogloss:0.73859
[14]	validation_0-mlogloss:0.73340
[15]	validation_0-mlogloss:0.72937
[16]	validation_0-mlogloss:0.72755
[17]	validation_0-mlogloss:0.72341
[18]	validation_0-mlogloss:0.720

[47]	validation_0-mlogloss:0.67983
[48]	validation_0-mlogloss:0.67832
[49]	validation_0-mlogloss:0.67676
training model for CV #5
[16:22:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:0.98161
[1]	validation_0-mlogloss:0.90895
[2]	validation_0-mlogloss:0.86210
[3]	validation_0-mlogloss:0.83243
[4]	validation_0-mlogloss:0.80506
[5]	validation_0-mlogloss:0.79099
[6]	validation_0-mlogloss:0.77373
[7]	validation_0-mlogloss:0.76070
[8]	validation_0-mlogloss:0.75111
[9]	validation_0-mlogloss:0.74228
[10]	validation_0-mlogloss:0.73428
[11]	validation_0-mlogloss:0.72731
[12]	validation_0-mlogloss:0.72455
[13]	validation_0-mlogloss:0.71883
[14]	validation_0-mlogloss:0.71595
[15]	validation_0-mlogloss:0.70994
[16]	vali

[44]	validation_0-mlogloss:0.69220
[45]	validation_0-mlogloss:0.69163
[46]	validation_0-mlogloss:0.69067
[47]	validation_0-mlogloss:0.69049
[48]	validation_0-mlogloss:0.68843
[49]	validation_0-mlogloss:0.68813
training model for CV #9
[16:22:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:0.99885
[1]	validation_0-mlogloss:0.93445
[2]	validation_0-mlogloss:0.89541
[3]	validation_0-mlogloss:0.84286
[4]	validation_0-mlogloss:0.80908
[5]	validation_0-mlogloss:0.79399
[6]	validation_0-mlogloss:0.78085
[7]	validation_0-mlogloss:0.76655
[8]	validation_0-mlogloss:0.75701
[9]	validation_0-mlogloss:0.74611
[10]	validation_0-mlogloss:0.74317
[11]	validation_0-mlogloss:0.73976
[12]	validation_0-mlogloss:0.73841
[13]	vali

[41]	validation_0-mlogloss:0.69947
[42]	validation_0-mlogloss:0.69927
[43]	validation_0-mlogloss:0.69933
[44]	validation_0-mlogloss:0.69900
[45]	validation_0-mlogloss:0.69865
[46]	validation_0-mlogloss:0.69932
[47]	validation_0-mlogloss:0.69931
[48]	validation_0-mlogloss:0.69972
[49]	validation_0-mlogloss:0.69974
training model for CV #13
[16:22:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.00086
[1]	validation_0-mlogloss:0.93927
[2]	validation_0-mlogloss:0.89612
[3]	validation_0-mlogloss:0.84667
[4]	validation_0-mlogloss:0.81359
[5]	validation_0-mlogloss:0.79834
[6]	validation_0-mlogloss:0.78361
[7]	validation_0-mlogloss:0.76940
[8]	validation_0-mlogloss:0.76326
[9]	validation_0-mlogloss:0.75442
[10]	val

[38]	validation_0-mlogloss:0.71485
[39]	validation_0-mlogloss:0.71385
[40]	validation_0-mlogloss:0.71358
[41]	validation_0-mlogloss:0.71313
[42]	validation_0-mlogloss:0.71273
[43]	validation_0-mlogloss:0.71312
[44]	validation_0-mlogloss:0.71266
[45]	validation_0-mlogloss:0.71115
[46]	validation_0-mlogloss:0.71060
[47]	validation_0-mlogloss:0.70824
[48]	validation_0-mlogloss:0.70767
[49]	validation_0-mlogloss:0.70791
training model for CV #17
[16:22:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:0.99970
[1]	validation_0-mlogloss:0.93822
[2]	validation_0-mlogloss:0.89780
[3]	validation_0-mlogloss:0.84788
[4]	validation_0-mlogloss:0.81943
[5]	validation_0-mlogloss:0.80267
[6]	validation_0-mlogloss:0.78799
[7]	v

In [30]:
from pathlib import Path
tst_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/tst')
val_dir = Path('C:/Users/Roh Seung Chan/Desktop/공모전/신용카드 예측 공모전/data/val')

algo_name = 'xgb'
feature_name = '2021'
model_name = f'{algo_name}_{feature_name}'

p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, xgb_p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, xgb_p_tst, fmt='%.6f', delimiter=',')

In [ ]:
sample_submission=pd.read_csv('data/sample_submission.csv')
for i in range(20):
    sample_submission.iloc[:,1:]+=models_xgb[i].predict_proba(test)/20
    
sample_submission.to_csv('state2021.csv',index=False)

In [3]:
feature_xgb=feature_xgb.sort_index()
feature_xgb

print(np.mean(np.array(logloss_avg)))

NameError: name 'feature_xgb' is not defined

In [30]:
print(np.mean(np.array(logloss_avg)))

0.6880214218100895


- 90 : 0.6890796591713713

In [28]:
label=train.credit
train0=train.drop('credit',axis=1)

skf=StratifiedKFold(n_splits=20,random_state=40,shuffle=True)

folds=[]
for train_idx,val_idx in skf.split(train0,label):
    folds.append([train_idx,val_idx])
    
models_xgb={}
logloss_avg=[]
feature_xgb=pd.DataFrame()

for i in tqdm(range(20)):
    train_x,val_x,train_y,val_y=train0.iloc[folds[i][0],:],train0.iloc[folds[i][1],:],label[folds[i][0]],label[folds[i][1]]
    xgb=XGBClassifier(max_depth=11,colsample_bytree=0.5,colsample_bylevel=0.8,colsample_bynode=0.5,min_child_weight=0.5,gamma=0,max_delta_step=0,subsample=0.9,reg_alpha=0.09,reg_lambda=0.6,n_estimators=100)
    xgb.fit(train_x,train_y)
    logloss_avg.append(log_loss(val_y,xgb.predict_proba(val_x)))
    models_xgb[i]=xgb
    feature_xgb=pd.concat([feature_xgb,pd.DataFrame(xgb.predict_proba(val_x),index=folds[i][1])],axis=0)
    
feature_xgb=feature_xgb.sort_index()
feature_xgb

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.748574
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.247245
[LightGBM] [Debug] init for col-wise cost 0.001282 seconds, init for row-wise cost 0.001238 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 1758
[LightGBM] [Info] Number of data points in the train set: 24900, number of used features: 20
[LightGBM] [Debug] Use subset for bagging
[LightGBM] [Info] Start training from score -2.105645
[LightGBM] [Info] Start training from score -1.440254
[LightGBM] [Info] Start training from score -0.444156
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [De

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Re-bagging, using 22402 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22422 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Deb

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22462 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22376 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 20
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22430 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22393 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22406 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Re-bagging, using 22414 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22472 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Re-bagging, using 22369 data to train
[LightGBM] [Debug] Traine

...

[LightGBM] [Debug] Re-bagging, using 22451 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22408 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22398 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Deb

[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Re-bagging, using 22358 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Re-bagging, using 22459 data to train
[LightGBM] [Debug] Traine

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22385 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 23
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Re-bagging, using 22443 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debu

[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 24
[LightGBM] [Debug] Re-bagging, using 22420 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 21
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 22
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 18
[LightGBM] [Debug] Re-bagging, using 22428 data to train
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 110 and depth = 16
[LightGBM] [Deb